In [1]:
import os

In [2]:
%pwd

'/Users/baldman/Documents/Deep Learning/End-to-End-Cancer-Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/baldman/Documents/Deep Learning/End-to-End-Cancer-Classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path =  PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = params_file_path

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [12]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config


    def download_file(self) -> str:
        """

        Fetch Data From Google Drive

        """

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} to file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} to file {zip_download_dir}")

        except Exception as e:
            raise e


    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir

        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-05 17:57:04,810: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-05 17:57:04,814: INFO: common: created directory at: artifacts]
[2025-02-05 17:57:04,815: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-05 17:57:04,816: INFO: 3570373505: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing to file artifacts/data_ingestion/data.zip]
Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=a46aedb9-fc66-49bd-a341-af583abab6e3
To: /Users/baldman/Documents/Deep Learning/End-to-End-Cancer-Classification/artifacts/data_ingestion/data.zip
100%|██████████| 49.0M/49.0M [00:06<00:00, 8.06MB/s]
[2025-02-05 17:57:15,154: INFO: 3570373505: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6